## Node 9

In [14]:
import numpy as np

# 수치미분 함수

def numerical_derivative(f, x):
    delta_x = 1e-4 # 0.0001
    grad = np.zeros_like(x)
    
    it = np.nditer(x, flags=['multi_index'], op_flags=['readwrite'])
    
    while not it.finished:
        idx = it.multi_index        
        tmp_val = x[idx]
        x[idx] = float(tmp_val) + delta_x
        fx1 = f(x) # f(x+delta_x)
        
        x[idx] = tmp_val - delta_x 
        fx2 = f(x) # f(x-delta_x)
        grad[idx] = (fx1 - fx2) / (2*delta_x)
        
        x[idx] = tmp_val 
        it.iternext()   
        
    return grad

# sigmoid 함수

def sigmoid(x):
    return 1 / (1+np.exp(-x))

In [24]:
class LogicGate:
        
    def __init__(self, gate_name, xdata, tdata):
        
        self.name = gate_name
        
        # 입력 데이터, 정답 데이터 초기화
        self.__xdata = xdata.reshape(4,2)  # 4개의 입력데이터 x1, x2 에 대하여 batch 처리 행렬
        self.__tdata = tdata.reshape(4,1)  # 4개의 입력데이터 x1, x2 에 대한 각각의 계산 값 행렬
        
        # 2층 hidden layer unit : 6개 가정,  가중치 W2, 바이어스 b2 초기화
        self.__W2 = np.random.rand(2,9)  # weight, 2 X 6 matrix
        self.__b2 = np.random.rand(9)
        
        # 3층 output layer unit : 1 개 , 가중치 W3, 바이어스 b3 초기화
        self.__W3 = np.random.rand(9,1)
        self.__b3 = np.random.rand(1)
                        
        # 학습률 learning rate 초기화
        self.__learning_rate = 1e-2
    
        print(self.name + " object is created")
            
    def feed_forward(self):        # feed forward 를 통하여 손실함수(cross-entropy) 값 계산
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z2 = np.dot(self.__xdata, self.__W2) + self.__b2  # 은닉층의 선형회귀 값
        a2 = sigmoid(z2)                                  # 은닉층의 출력
        
        z3 = np.dot(a2, self.__W3) + self.__b3            # 출력층의 선형회귀 값
        y = a3 = sigmoid(z3)                              # 출력층의 출력
    
        # cross-entropy 
        return  -np.sum( self.__tdata*np.log(y + delta) + (1-self.__tdata)*np.log((1 - y)+delta ) )    
    
    def loss_val(self):          # 외부 출력을 위한 손실함수(cross-entropy) 값 계산 
        
        delta = 1e-7    # log 무한대 발산 방지
    
        z2 = np.dot(self.__xdata, self.__W2) + self.__b2  # 은닉층의 선형회귀 값
        a2 = sigmoid(z2)                                  # 은닉층의 출력
        
        z3 = np.dot(a2, self.__W3) + self.__b3            # 출력층의 선형회귀 값
        y = a3 = sigmoid(z3)                              # 출력층의 출력
    
        # cross-entropy 
        return  -np.sum( self.__tdata*np.log(y + delta) + (1-self.__tdata)*np.log((1 - y)+delta ) )
    
    
    # 수치미분을 이용하여 손실함수가 최소가 될때 까지 학습하는 함수
    def train(self):
        
        f = lambda x : self.feed_forward()
        
        print("Initial loss value = ", self.loss_val())
        
        for step in  range(10001):
            
            self.__W2 -= self.__learning_rate * numerical_derivative(f, self.__W2)
    
            self.__b2 -= self.__learning_rate * numerical_derivative(f, self.__b2)
        
            self.__W3 -= self.__learning_rate * numerical_derivative(f, self.__W3)
    
            self.__b3 -= self.__learning_rate * numerical_derivative(f, self.__b3)
    
            if (step % 400 == 0):
                print("\n*****************************************************************************")
                print("step = ", step, "  , loss value = ", self.loss_val())
                print("\nW2 = " , self.__W2, "\n")
                print("b2 = " , self.__b2, "\n")
                
    
    # query, 즉 미래 값 예측 함수
    def predict(self, xdata):
        
        z2 = np.dot(xdata, self.__W2) + self.__b2         # 은닉층의 선형회귀 값
        a2 = sigmoid(z2)                                  # 은닉층의 출력
        
        z3 = np.dot(a2, self.__W3) + self.__b3            # 출력층의 선형회귀 값
        y = a3 = sigmoid(z3)                              # 출력층의 출력
    
        if y > 0.5:
            result = 1  # True
        else:
            result = 0  # False
    
        return y, result

In [25]:
# AND Gate 객체 생성 및 training

xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([0, 0, 0, 1])

and_obj = LogicGate("AND", xdata, tdata)

and_obj.train()

AND object is created
Initial loss value =  12.923866368400887

*****************************************************************************
step =  0   , loss value =  12.403709816362635

W2 =  [[0.62139613 0.25276129 0.37701564 0.78233114 0.67740294 0.40889123
  0.96780086 0.81354786 0.18239215]
 [0.81121706 0.90484247 0.89146185 0.1511099  0.94128184 0.83555085
  0.3725448  0.26873639 0.43105563]] 

b2 =  [0.89949767 0.83876021 0.50987437 0.90626097 0.53263992 0.57009842
 0.23364936 0.53575235 0.71018036] 


*****************************************************************************
step =  400   , loss value =  2.178360340432476

W2 =  [[0.60162287 0.24047871 0.39325992 0.72314257 0.67450993 0.4098
  0.95592984 0.76133998 0.1467767 ]
 [0.79160402 0.90599049 0.9260757  0.11369156 0.94428426 0.84618772
  0.35879184 0.23235382 0.39677052]] 

b2 =  [0.91572308 0.74047481 0.30455416 0.9620139  0.36227614 0.43495781
 0.20317446 0.58729619 0.673433  ] 


*******************************


*****************************************************************************
step =  7200   , loss value =  0.08560183247005437

W2 =  [[ 0.52791762  0.31091312  1.86098256 -1.20056455  1.91636736  1.08685274
   2.02569441  0.30690175 -1.5435981 ]
 [ 0.69331199  1.07028609  2.16191098 -1.43006014  1.99794068  1.54780775
   1.10939955 -0.06128132 -1.2740705 ]] 

b2 =  [ 1.43050544  0.39139437 -2.7536182   1.66770637 -2.67014739 -1.50221414
 -1.92357112  0.95053534  1.74239678] 


*****************************************************************************
step =  7600   , loss value =  0.07689746672218348

W2 =  [[ 0.5280251   0.31077926  1.89152256 -1.22712561  1.94344711  1.102405
   2.03968999  0.29824382 -1.5657604 ]
 [ 0.69234408  1.07177766  2.18586221 -1.45516061  2.02411306  1.56201365
   1.13322987 -0.06800554 -1.30016838]] 

b2 =  [ 1.43773861  0.38777501 -2.79868072  1.70046235 -2.71412309 -1.53401519
 -1.96061284  0.95474071  1.7837234 ] 


*******************************

In [26]:
# AND Gate prediction

test_data = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])

for data in test_data:
    print(and_obj.predict(data))

(array([5.56577322e-05]), 0)
(array([0.01213451]), 0)
(array([0.01225424]), 0)
(array([0.97865927]), 1)


In [27]:
# OR Gate 객체 생성 및 training

xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([0, 1, 1, 1])

or_obj = LogicGate("OR", xdata, tdata)

or_obj.train()

OR object is created
Initial loss value =  3.9423339785817038

*****************************************************************************
step =  0   , loss value =  3.9012113672303395

W2 =  [[0.69938422 0.18218233 0.9230102  0.35352237 0.44683821 0.66167196
  0.12547001 0.01577447 0.05249773]
 [0.35263152 0.19439866 0.98026817 0.97581691 0.2530963  0.34564395
  0.22636684 0.73427797 0.4612407 ]] 

b2 =  [0.32376135 0.67583941 0.46103227 0.10154547 0.45119337 0.86862795
 0.78690648 0.51513479 0.16699078] 


*****************************************************************************
step =  400   , loss value =  1.9990691790628714

W2 =  [[0.68963341 0.30514712 0.8842408  0.46877015 0.55735775 0.7064195
  0.26546201 0.17121991 0.1734417 ]
 [0.34271349 0.30920301 0.94541523 1.06993804 0.3605529  0.39218208
  0.35433209 0.85808437 0.56972814]] 

b2 =  [ 0.32018499  0.52964898  0.55183669 -0.04229879  0.3079237   0.76640182
  0.62195179  0.32407493  0.04687974] 


*******************


*****************************************************************************
step =  7200   , loss value =  0.0813911017531977

W2 =  [[0.71723088 0.54980981 1.15821777 3.57202935 2.04531422 0.74500967
  0.58034863 2.34035643 1.70342575]
 [0.36458706 0.52424329 1.17703651 3.54789962 1.76822868 0.42436107
  0.62507418 2.51101933 1.83300981]] 

b2 =  [ 0.31657922  0.36043127  0.01822418 -1.9140339  -1.07046141  0.76061508
  0.366975   -1.35962803 -1.01136495] 


*****************************************************************************
step =  7600   , loss value =  0.07405353647064356

W2 =  [[0.71294594 0.54543981 1.16349346 3.61600415 2.07316363 0.73936957
  0.57704202 2.378865   1.73121776]
 [0.35996894 0.52008174 1.18195544 3.58991795 1.79738626 0.41800958
  0.6220163  2.54438368 1.85799487]] 

b2 =  [ 0.3227435   0.36638933  0.00759263 -1.93503846 -1.08998688  0.77165086
  0.37176882 -1.37932107 -1.02888379] 


******************************************************************

In [28]:
# OR Gate prediction

test_data = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])

for data in test_data:
    print(or_obj.predict(data))

(array([0.02756317]), 0)
(array([0.99113129]), 1)
(array([0.99066075]), 1)
(array([0.99931572]), 1)


In [29]:
# NAND Gate 객체 생성 및 training

xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([1, 1, 1, 0])

nand_obj = LogicGate("NAND", xdata, tdata)

nand_obj.train()

NAND object is created
Initial loss value =  6.366864371766156

*****************************************************************************
step =  0   , loss value =  6.294157734556809

W2 =  [[0.26212023 0.57671022 0.91996226 0.24415119 0.95847566 0.47253312
  0.48583167 0.75136002 0.12224776]
 [0.51456366 0.38095801 0.63979229 0.51827438 0.0657543  0.18446193
  0.76268146 0.41449544 0.85756536]] 

b2 =  [0.09771477 0.55055759 0.66090974 0.84625164 0.52266281 0.47247972
 0.8914636  0.76011667 0.35722047] 


*****************************************************************************
step =  400   , loss value =  2.3309606223381545

W2 =  [[ 0.13634083  0.51265663  0.90935237  0.16221986  0.79503654  0.42533285
   0.42727776  0.65919948 -0.03965852]
 [ 0.38724943  0.31822366  0.62710612  0.43239162 -0.0763056   0.13776867
   0.6994884   0.32953714  0.68118379]] 

b2 =  [0.04933343 0.52237886 0.61760654 0.82667101 0.50739224 0.43658834
 0.89858312 0.76576289 0.32640884] 


*********


*****************************************************************************
step =  6800   , loss value =  0.1158872510858707

W2 =  [[-0.25380315  1.09754623  2.42857234 -1.70983345 -0.87528988  0.84032664
   0.43530406  0.59694442 -1.69009372]
 [-0.02989628  0.92302121  2.39533441 -1.68393359 -1.80643325  0.53214223
   0.74259124  0.27162896 -1.14094178]] 

b2 =  [ 0.11128794 -0.60401778 -3.29472282  2.09110844  1.35287917  0.06242588
  0.82498975  0.83732505  1.53513479] 


*****************************************************************************
step =  7200   , loss value =  0.10200868362555604

W2 =  [[-0.26311475  1.11160778  2.47307338 -1.74376404 -0.90139369  0.85031781
   0.43686345  0.59940579 -1.7166797 ]
 [-0.03908262  0.93510391  2.44039976 -1.71752107 -1.83123564  0.53953946
   0.74543604  0.27328681 -1.16962896]] 

b2 =  [ 0.11186372 -0.63711737 -3.36618034  2.1491812   1.39727168  0.0490795
  0.81985881  0.83512946  1.58650311] 


*******************************

In [30]:
# NAND Gate prediction

test_data = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])

for data in test_data:
    print(nand_obj.predict(data))

(array([0.99993455]), 1)
(array([0.98687471]), 1)
(array([0.98700065]), 1)
(array([0.02576904]), 0)


In [31]:
# XOR Gate 객체 생성

xdata = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])
tdata = np.array([0, 1, 1, 0])


xor_obj = LogicGate("XOR", xdata, tdata)

xor_obj.train()

XOR object is created
Initial loss value =  6.5538693869156965

*****************************************************************************
step =  0   , loss value =  6.367475142074479

W2 =  [[0.45290599 0.20554626 0.45828699 0.98708637 0.64928909 0.32432905
  0.82113284 0.74575112 0.38692808]
 [0.50548379 0.04382995 0.25967212 0.93699938 0.50442005 0.1931032
  0.78911421 0.48701335 0.95413677]] 

b2 =  [0.14849616 0.62098052 0.54910219 0.19563137 0.08073352 0.54975914
 0.70809689 0.5189335  0.26136182] 


*****************************************************************************
step =  400   , loss value =  2.770488221197539

W2 =  [[0.43926697 0.20146092 0.4481555  0.97334733 0.6420904  0.31233223
  0.8378266  0.72484507 0.39380283]
 [0.49405993 0.0371565  0.24834376 0.92301908 0.49799956 0.18133493
  0.80604192 0.45417391 0.94914531]] 

b2 =  [0.14307663 0.61701231 0.53108675 0.18652982 0.05326126 0.52277949
 0.66835964 0.52901856 0.22936759] 


*****************************


*****************************************************************************
step =  7200   , loss value =  1.6004234823278436

W2 =  [[0.5626451  0.67416921 0.73070276 3.40407658 0.79418348 0.58440593
  3.43403034 1.04007497 0.543209  ]
 [1.10219123 0.38346804 0.41381657 3.32202379 0.74924862 0.49896446
  3.35424002 0.24189321 1.06970766]] 

b2 =  [-0.60261455  0.34539906  0.18608323 -0.40808091 -0.46385939  0.23119817
 -0.42496738  0.07471124  0.1123348 ] 


*****************************************************************************
step =  7600   , loss value =  1.4376768536204518

W2 =  [[0.62957843 0.72682871 0.78318309 3.58758022 0.84862555 0.62353929
  3.62335681 1.13004304 0.51172748]
 [1.23089203 0.41864052 0.45067212 3.50726881 0.82860876 0.54883015
  3.54499726 0.25447978 1.08818684]] 

b2 =  [-0.81470987  0.24844117  0.08398111 -0.51349897 -0.6250759   0.13820135
 -0.53691403 -0.04245204  0.03207067] 


*******************************************************************

In [32]:
test_data = np.array([ [0, 0], [0, 1], [1, 0], [1, 1] ])

for data in test_data:
    print(xor_obj.predict(data))

(array([0.08732785]), 0)
(array([0.87132322]), 1)
(array([0.85988041]), 1)
(array([0.18982677]), 0)
